### --- Day 15: Lens Library ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/15

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 15);
WriteLines(inputLines, maxCols: 50);

Loading puzzle file: Day15.txt
Total lines: 1
Max line length: 22928

fb-,kgf=2,xdr-,fqqg=7,cgth-,hkpjg-,gl=7,nrq-,jp-,r


In [4]:
Console.WriteLine(Encoding.ASCII.GetBytes("A")[0]);
Console.WriteLine(byte.MaxValue * 17);

65
4335


In [5]:
// Determine the ASCII code for the current character of the string.
// Increase the current value by the ASCII code you just determined.
// Set the current value to itself multiplied by 17.
// Set the current value to the remainder of dividing itself by 256.

int HashOne(int previous, int next) => (int)((previous + next) * 17 % 256);

In [6]:
byte Hash(string input) => (byte)Encoding.ASCII.GetBytes(input).Select(b => (int)b).Aggregate(0, HashOne);

In [7]:
const string testInputLine = "rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7";

var hashSteps = testInputLine.Split(',').Select(Hash);
Console.WriteLine(string.Join(",", hashSteps));

30,253,97,47,14,180,9,197,48,214,231


In [8]:
// Run the HASH algorithm on each step in the initialization sequence. What is
// the sum of the results? (The initialization sequence is one long line; be
// careful when copy-pasting it.)

var part1Answer = inputLines[0].Split(',').Select(input => (int)Hash(input)).Sum();
Console.WriteLine(part1Answer);

513158


In [9]:
// 513158 is correct!
Ensure(513158, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/15

In [11]:
record Lens(string label, byte focalLength);

interface LensAction { byte boxHash {get;} }
record LensAdd(byte boxHash, Lens lens) : LensAction;
record LensRemove(byte boxHash, string label) : LensAction;

LensAction ParseAction(string action) 
{
    LensAction result;
    var maybeAdd = action.Split('=');
    if (maybeAdd.Length == 2) {
        var boxHash = Hash(maybeAdd[0]);
        result = new LensAdd(boxHash, new Lens(maybeAdd[0], byte.Parse(maybeAdd[1])));
    } else {
        var removeLabel = action.Substring(0, action.Length - 1);
        var boxHash = Hash(removeLabel);
        result = new LensRemove(boxHash, removeLabel);
    }

    return result;
}

var testInputActions = testInputLine.Split(',').Select(ParseAction).ToList();
foreach (var action in testInputActions) {
    Console.WriteLine(action);
}

LensAdd { boxHash = 0, lens = Lens { label = rn, focalLength = 1 } }
LensRemove { boxHash = 0, label = cm }
LensAdd { boxHash = 1, lens = Lens { label = qp, focalLength = 3 } }
LensAdd { boxHash = 0, lens = Lens { label = cm, focalLength = 2 } }
LensRemove { boxHash = 1, label = qp }
LensAdd { boxHash = 3, lens = Lens { label = pc, focalLength = 4 } }
LensAdd { boxHash = 3, lens = Lens { label = ot, focalLength = 9 } }
LensAdd { boxHash = 3, lens = Lens { label = ab, focalLength = 5 } }
LensRemove { boxHash = 3, label = pc }
LensAdd { boxHash = 3, lens = Lens { label = pc, focalLength = 6 } }
LensAdd { boxHash = 3, lens = Lens { label = ot, focalLength = 7 } }


In [12]:
class LensState
{
    Dictionary<byte, List<Lens>> _boxDict = new();

    List<Lens> GetBox(byte boxHash) {
        List<Lens> box;
        if (!_boxDict.TryGetValue(boxHash, out box)) {
            box = new();
            _boxDict[boxHash] = box;
        }

        return box;
    }

    public LensState Process(LensAction action) => action switch {
        (LensAdd add) => ProcessAdd(add),
        (LensRemove remove) => ProcessRemove(remove),
        _ => throw new Exception("Unexpected action type")
    };

    LensState ProcessAdd(LensAdd add) {
        var box = GetBox(add.boxHash);

        bool swapped = false;
        for (var i = 0; i < box.Count; i++) {
            var existing = box[i];

            if (existing.label == add.lens.label) {
                box[i] = add.lens;
                swapped = true;
            }
        }

        if (!swapped) {
            box.Add(add.lens);
        }

        return this;
    }

    LensState ProcessRemove(LensRemove remove) {
        var box = GetBox(remove.boxHash);

        var newBox = box.Where(l => l.label != remove.label).ToList();
        box.Clear();
        box.AddRange(newBox);

        return this;
    }

    public void WriteToConsole() {
        for (var i = 0; i < 256; i++) {
            var box = GetBox((byte)i);

            if (box.Count > 0) {
                Console.Write($"Box {i}: ");
                Console.WriteLine(string.Join(",", box));
            }
        }
    }

    public int FocusingPower {
        get {
            int result = 0;

            foreach (var i in Enumerable.Range(0, 256)) {
                var box = GetBox((byte)i);

                var j = 0;
                foreach (var lens in box) {
                    // One plus the box number of the lens in question.
                    var a = i + 1;
                    // The slot number of the lens within the box: 1 for the first lens, 2 for the second lens, and so on.
                    var b = ++j;
                    // The focal length of the lens.
                    var c = lens.focalLength;

                    result += a * b * c;
                }
            }

            return result;
        }
    }
}

var testInputLensState = testInputActions.Aggregate(new LensState(), (state, action) => state.Process(action));

// After "ot=7":
// Box 0: [rn 1] [cm 2]
// Box 3: [ot 7] [ab 5] [pc 6]
testInputLensState.WriteToConsole();

// So, the above example ends up with a total focusing power of 145.
Console.WriteLine(testInputLensState.FocusingPower);


Box 0: Lens { label = rn, focalLength = 1 },Lens { label = cm, focalLength = 2 }
Box 3: Lens { label = ot, focalLength = 7 },Lens { label = ab, focalLength = 5 },Lens { label = pc, focalLength = 6 }
145


In [13]:
// With the help of an over-enthusiastic reindeer in a hard hat, follow the
// initialization sequence. What is the focusing power of the resulting lens
// configuration?

var inputActions = inputLines[0].Split(',').Select(ParseAction);
var lensState = inputActions.Aggregate(new LensState(), (state, action) => state.Process(action));
var part2Answer = lensState.FocusingPower;
Console.WriteLine(part2Answer);

200277


In [14]:
// 200277 is correct!
Ensure(200277, part2Answer);